## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-02-02-43-42 +0000,nyt,Oscar-Nominated Screenwriter Arrested in Iran ...,https://www.nytimes.com/2026/02/01/world/middl...
1,2026-02-02-02-29-58 +0000,startribune,Timberwolves’ Anthony Edwards named NBA All-St...,https://www.startribune.com/timberwolves-antho...
2,2026-02-02-02-27-00 +0000,wsj,China Manufacturing Gauge Shows Pickup in Acti...,https://www.wsj.com/economy/china-manufacturin...
3,2026-02-02-02-23-12 +0000,startribune,Live: Musicians call out ICE actions in Minnes...,https://www.startribune.com/ice-raids-minnesot...
4,2026-02-02-02-13-11 +0000,nypost,"Timothy Busfield once branded ‘creep,’ accused...",https://nypost.com/2026/02/01/us-news/timothy-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2506/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
70,trump,26
167,epstein,15
26,ice,13
4,iran,12
15,year,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
20,2026-02-02-00-53-49 +0000,nypost,Trump announces two-year closure of Trump Kenn...,https://nypost.com/2026/02/01/us-news/trump-an...,83
197,2026-02-01-04-26-00 +0000,wsj,President Trump joked during a speech that he ...,https://www.wsj.com/politics/policy/trump-joke...,73
131,2026-02-01-15-09-48 +0000,nypost,Trump jokes about suing new Fed chair nominee ...,https://nypost.com/2026/02/01/us-news/trump-jo...,67
8,2026-02-02-01-44-38 +0000,nyt,Trump Says Kennedy Center Will Close for 2-Yea...,https://www.nytimes.com/2026/02/01/us/politics...,60
58,2026-02-01-22-34-48 +0000,nypost,India to buy oil from Venezuela rather than Ru...,https://nypost.com/2026/02/01/world-news/india...,59


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
20,83,2026-02-02-00-53-49 +0000,nypost,Trump announces two-year closure of Trump Kenn...,https://nypost.com/2026/02/01/us-news/trump-an...
197,47,2026-02-01-04-26-00 +0000,wsj,President Trump joked during a speech that he ...,https://www.wsj.com/politics/policy/trump-joke...
146,44,2026-02-01-13-52-32 +0000,nypost,Ex-Prince Andrew’s creepy reply to Jeffrey Eps...,https://nypost.com/2026/02/01/world-news/ex-pr...
85,41,2026-02-01-20-45-02 +0000,startribune,Johnson says no quick House vote to end partia...,https://www.startribune.com/johnson-says-no-qu...
89,38,2026-02-01-20-12-26 +0000,nypost,Gov. Hochul will raise New York’s taxes after ...,https://nypost.com/2026/02/01/us-news/cowardly...
34,32,2026-02-02-00-07-50 +0000,nypost,Bizarre origins of Gavin Newom’s ‘look’: Gover...,https://nypost.com/2026/02/01/us-news/gavin-ne...
168,32,2026-02-01-10-54-14 +0000,nypost,Iran’s supreme leader warns any US attack woul...,https://nypost.com/2026/02/01/world-news/iran-...
199,29,2026-02-01-04-18-45 +0000,nyt,Judge Orders Release of 5-Year-Old Detained by...,https://www.nytimes.com/2026/01/31/us/politics...
18,26,2026-02-02-00-55-00 +0000,wsj,JGBs Fall Amid Prospects That Japan’s Ruling P...,https://www.wsj.com/finance/investing/jgbs-fal...
74,24,2026-02-01-21-34-42 +0000,nypost,"PENN District poised to grow by another block,...",https://nypost.com/2026/02/01/business/penn-di...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
